In [ ]:
!pip install riotwatcher

In [2]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd

In [58]:
 TIERS = ["IRON", "SILVER", "GOLD", "PLATINUM", "DIAMOND"]
RANKS = ["I", "II", "III", "IV"]
REGIONS = ["eun1"]
PAGES = [1]

In [83]:
class Karma:
  def __init__(self, API_KEY):
    self.API_KEY = API_KEY
    self.lol_watcher = LolWatcher(API_KEY)

  def player_stats(self, region, summoner):
    player = self.lol_watcher.summoner.by_name(region, summoner)
    ranked_stats = self.lol_watcher.league.by_summoner(region, player['id'])
    columns = [
        "name",
        "summonerLevel",
        "tierRank",
        "wins",
        "losses",
        "veteran",
        "inactive",
        "freshBlood",
        "hotStreak"
    ]

    data = {}
    # player = lol_watcher.summoner.by_name('eun1', 'spikal')
    data['name'] = player['name']
    data['summonerLevel'] = player['summonerLevel']
    if len(ranked_stats) > 0:
      for i in range(0,len(ranked_stats)):
        if 'RANKED_SOLO_5x5' in ranked_stats[i]['queueType']:
          if 'tier' in ranked_stats[i] and 'rank' in ranked_stats[i]:
            t = ranked_stats[i]['tier'] + ' ' + ranked_stats[i]['rank']
            data['tierRank'] = t
          else:
            data['tierRank'] = None

          if 'wins' in ranked_stats[i]:
            t = ranked_stats[i]['wins']
            data['wins'] = t
          else:
            data['wins'] = None

          if 'losses' in ranked_stats[i]:
            t = ranked_stats[i]['losses']
            data['losses'] = t
          else:
            data['losses'] = None

          if 'veteran' in ranked_stats[i]:
            t = ranked_stats[i]['veteran']
            data['veteran'] = t
          else:
            data['veteran'] = None

          if 'inactive' in ranked_stats[i]:
            t = ranked_stats[i]['inactive']
            data['inactive'] = t
          else:
            data['inactive'] = None

          if 'freshBlood' in ranked_stats[i]:
            t = ranked_stats[i]['freshBlood']
            data['freshBlood'] = t
          else:
            data['freshBlood'] = None

          if 'hotStreak' in ranked_stats[i]:
            t = ranked_stats[i]['hotStreak']
            data['hotStreak'] = t
          else:
            data['hotStreak'] = None
    else:
      data['tierRank'] = None
      data['wins'] = None
      data['losses'] = None
      data['veteran'] = None
      data['inactive'] = None
      data['freshBlood'] = None
      data['hotStreak'] = None
    return data

In [86]:
api_key= "RGAPI-c7cf2424-731f-4a9a-8f1f-f6985e62dac3"
karma = Karma(api_key)
data = karma.player_stats(region = 'eun1', summoner='spikal')

In [87]:
data

{'name': 'Spikal',
 'summonerLevel': 491,
 'tierRank': 'EMERALD IV',
 'wins': 97,
 'losses': 88,
 'veteran': False,
 'inactive': False,
 'freshBlood': False,
 'hotStreak': False}